In [1]:
import asyncio
from ag_motioncontrol import MotionController
from ag_motioncontrol.generated_python_api.pyactiongraph.launcher import APIBase

In [2]:
APIBase.simulator_port_range_start = 15000
ide_host = "127.0.0.1"
connection = "simulation"
ide_client_conn_port = 20000
ide_sim_conn_port = 30000
ide_host = "127.0.0.1"
api = MotionController("robot_01", manual_transport_config=connection, force_reset=False, 
                    debug_log_level=0,
                    additional_client_rpc_connections=[f'zmq in=tcp://{ide_host}:{ide_client_conn_port + 1} out=tcp://0.0.0.0:{ide_client_conn_port}'],
                    additional_simulator_rpc_connections=[f'zmq in=tcp://{ide_host}:{ide_sim_conn_port + 1} out=tcp://0.0.0.0:{ide_sim_conn_port}'])


In [3]:
async def actiongraph_test():
        
    test_movements = [
        {
            "position": 0.1,
            "axis": 1,
        },
        {
            "position": 0.15,
            "axis": 2,
        },
        {
            "position": 0.05,
            "axis": 1,
        },
        {
            "position": 0.05,
            "axis": 2,
        }  
    ]
    await api.start()
    try:
        await api.wait_for_ready()
        for i in range(1, 3):
            await api.set_pos_feedback_frequency(i, 50)

        await asyncio.gather(*[api.do_homing(i, speed=0.1) for i in range(1, 3)])

        for  _ in range(4):
            await api.execute_movements(test_movements, max_acceleration=0.5)
            for i in range(1, 3):
                print(f"Axes {i} position: {api.get_axis_pos(i)}")
     
    finally:
        await api.stop()
t = asyncio.create_task(actiongraph_test())
await t

Axes 1 position: 0.05001933127641678
Axes 2 position: 0.05001933127641678
Axes 1 position: 0.049996208399534225
Axes 2 position: 0.049973081797361374
Axes 1 position: 0.049973081797361374
Axes 2 position: 0.049973081797361374
Axes 1 position: 0.049973081797361374
Axes 2 position: 0.049973081797361374
Shutting down simulator robot(robot_01)
Shutted down simulator robot(robot_01)
